In [ ]:
import bn3d
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import hashlib
import json
import binascii
from bn3d.utils import NumpyEncoder, hash_json
from bn3d.statmech.rbim2d import (
    RandomBondIsingModel2D, Magnetization,
    Rbim2DIidDisorder
)
from bn3d.statmech.controllers import SimpleController
from bn3d.statmech.analysis import SimpleAnalysis, load_analysis
import uuid
import pandas as pd
import cProfile
from pstats import Stats
import os
import psutil
from itertools import product

## Load data

In [ ]:
data_dir = '../temp/statmech/test_7/'

In [ ]:
analysis = load_analysis(data_dir)

In [ ]:
estimates = analysis.estimates

In [ ]:
max_tau = estimates['tau'].max()

## Plot single observable

In [ ]:
label = 'CorrelationRatio'
for p_val in estimates['p'].unique():
    for L_x_val in estimates['L_x'].unique():
        estimates_filt = estimates[
            (estimates['p'] == p_val)
            & (estimates['tau'] == max_tau)
            & (estimates['L_x'] == L_x_val)
        ][['temperature', f'{label}_estimate', f'{label}_uncertainty']]
        y_values = np.real(estimates_filt[f'{label}_estimate'])
        y_values[y_values == 0] = np.nan
        y_err = np.real(estimates_filt[f'{label}_uncertainty'])
        plt.errorbar(
            estimates_filt['temperature'],
            y_values,
            yerr=y_err,
            fmt='o-',
            capsize=5,
            label=f'L = {L_x_val}'
        )
    plt.ylabel(r'Correlation Ratio $\xi/L$')
    plt.xlabel('Temperature $T$')
    plt.title(f'p = {p_val}')
    plt.yscale('log')
    plt.legend()
    plt.show()

## Plot multiple observable

In [ ]:
observable_list = [
    'Energy', 'SpecificHeat', 'Magnetization', 'Magnetization_2',
    'Susceptibility0', 'Susceptibilitykmin', 'CorrelationRatio',
    'Binder'
]
for label in observable_list:
    for p_val in estimates['p'].unique():
        for L_x_val in estimates['L_x'].unique():
            estimates_filt = estimates[
                (estimates['p'] == p_val)
                & (estimates['tau'] == max_tau)
                & (estimates['L_x'] == L_x_val)
            ][['temperature', f'{label}_estimate', f'{label}_uncertainty']]
            # display(estimates_filt[['temperature']])
            y_values = np.real(estimates_filt[f'{label}_estimate'])
            y_values[y_values == 0] = np.nan
            y_err = np.real(estimates_filt[f'{label}_uncertainty'])
            plt.errorbar(
                estimates_filt['temperature'],
                y_values,
                yerr=y_err,
                fmt='o-',
                capsize=5,
                label=f'L = {L_x_val}'
            )
        plt.ylabel(label)
        plt.xlabel('temperature')
        plt.title(f'p = {p_val}')
        plt.legend()
        plt.show()

## Plot Wilson loops

In [ ]:
p_val = estimates['p'].unique()[0]
L_x_val = 10


wilson_loops = {}
for T_val in estimates['temperature'].unique():
    filt_estimates = estimates[(estimates['p'] == p_val) & 
                               (estimates['tau'] == max_tau) &
                               (estimates['L_x'] == L_x_val) &
                               (estimates['temperature'] == T_val)]
    wilson_loops[T_val] = filt_estimates['Wilson Loop_estimate'].to_numpy()[0]
    
n_wilson_loops = len(list(wilson_loops.values())[0])

In [ ]:
plt.rcParams['figure.figsize'] = (7, 5)
plt.rcParams['font.size'] = 15

eps = 1e-5
for i, p_val in enumerate(estimates['p'].unique()):
    plt.subplot(len(estimates['p'].unique()), 1, i+1)
    for T_val in estimates['temperature'].unique():
        filt_estimates = estimates[(estimates['p'] == p_val) & 
                                   (estimates['tau'] == max_tau) &
                                   (estimates['L_x'] == L_x_val) &
                                   (estimates['temperature'] == T_val)]
        wilson_loops = filt_estimates['Wilson Loop_estimate'].to_numpy()[0]
        n_wilson_loops = len(wilson_loops)
        
        areas = np.arange(2, n_wilson_loops+1)**2
        perimeters = 4 * (np.arange(2, n_wilson_loops+1) - 1)

        log_wl = -np.log((1+wilson_loops[1:])/2+eps) / perimeters

        plt.plot(perimeters, log_wl, 'o-', label=f'T={T_val}');

    plt.title(f'p={p_val}')
    plt.ylabel("$-\log(WL) / P$")
    plt.xlabel("Perimeter")
    plt.xticks(perimeters)
    plt.legend()